In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [2]:
# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')
# データフレームを表示して確認
df.head()

,カテゴリー,商品コード,商品名,売上日,単価,数量,原価
0,食品,1001,りんご,2023-01-01,200,50,120
1,食品,1002,バナナ,2023-01-01,150,100,80
2,食品,1003,牛乳,2023-01-02,180,80,100
3,衣服,2001,Tシャツ,2023-01-02,1500,20,800
4,衣服,2002,ジーンズ,2023-01-03,5000,10,2500


In [3]:
# 2. データをLLM用にテキスト形式に変換
# データフレーム全体を文字列に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"
# 表示して確認
print(prompt_text)

売上データ:
    カテゴリー 商品コード      商品名         売上日    単価   数量    原価
0      食品  1001      りんご  2023-01-01   200   50   120
1      食品  1002      バナナ  2023-01-01   150  100    80
2      食品  1003       牛乳  2023-01-02   180   80   100
3      衣服  2001     Tシャツ  2023-01-02  1500   20   800
4      衣服  2002     ジーンズ  2023-01-03  5000   10  2500
..    ...   ...      ...         ...   ...  ...   ...
235    衣服  2077   レインパンツ  2023-04-28  2000   18  1000
236    食品  1085      ザクロ  2023-04-29   600   40   300
237   日用品  3077    バスブラシ  2023-04-29   400   60   200
238    衣服  2078  レインシューズ  2023-04-30  2500   15  1250
239    食品  1086    ココナッツ  2023-04-30   300   80   150

[240 rows x 7 columns]
この売上データの傾向を分析してください。


In [4]:
# 3. OpenAI APIの呼び出し

# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# LLMからの回答を表示
print(response.choices[0].message.content.strip())

売上データから傾向を分析するためには、いくつかの重要なポイントを考慮する必要があります。以下は、データの傾向を分析する際の主要な観点と、いくつかの示唆です。

### 1. カテゴリー別売上分析
- **売上高**: 各カテゴリー（食品、衣服、日用品）ごとの売上高を集計し、どのカテゴリーが最も売上を上げているかを特定します。
- **利益率**: 各商品の単価と原価から利益を計算し、どのカテゴリーが最も高い利益率を持っているか分析します。

### 2. 商品別パフォーマンス
- **売上ランキング**: 単価、数量、売上高で商品別にランキングを作成し、人気の商品を特定します。
- **在庫管理**: 売上数量に基づいて在庫の適切な水準を確保するための分析を行います。特に、人気商品については在庫が不足しないように注意します。

### 3. 時間的傾向
- **売上の季節性**: 売上日を基に月ごとの売上推移を分析し、季節性のトレンドを見つけます。特に閑散期とピーク期を特定します。
- **プロモーション効果**: 特定の日や月にプロモーションを行った場合、売上にどのような影響があったかを分析します。プロモーション前後の売上を比較します。

### 4. 顧客の購買行動
- **頻繁に購入される商品**: リピート率が高い商品を特定し、その商品に対するマーケティング戦略を強化します。
- **セット販売の機会**: しばしば一緒に購入される商品のパターンを見つけ、バンドル販売やセット販売の機会を探ります。

### 5. その他の洞察
- **競合分析**: 他の競合と比較して、自社商品がどのように位置付けられているかを確認します。競合商品の価格や特徴と比較することで、自社の競争力を評価します。

### 結論
これらの観点から売上データを緻密に分析することで、企業は効率的にリソースを配分し、マーケティング戦略を最適化することができます。データの詳細を分析し、実際の数値を取得することで、さらに具体的で実行可能なインサイトを導出することが可能になります。


In [5]:
# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])
print(df_out)

                                                   結果
0   売上データから傾向を分析するためには、いくつかの重要なポイントを考慮する必要があります。以下...
1                                                    
2                                   ### 1. カテゴリー別売上分析
3   - **売上高**: 各カテゴリー（食品、衣服、日用品）ごとの売上高を集計し、どのカテゴリー...
4   - **利益率**: 各商品の単価と原価から利益を計算し、どのカテゴリーが最も高い利益率を持...
5                                                    
6                                   ### 2. 商品別パフォーマンス
7   - **売上ランキング**: 単価、数量、売上高で商品別にランキングを作成し、人気の商品を特...
8   - **在庫管理**: 売上数量に基づいて在庫の適切な水準を確保するための分析を行います。特...
9                                                    
10                                       ### 3. 時間的傾向
11  - **売上の季節性**: 売上日を基に月ごとの売上推移を分析し、季節性のトレンドを見つけま...
12  - **プロモーション効果**: 特定の日や月にプロモーションを行った場合、売上にどのような...
13                                                   
14                                     ### 4. 顧客の購買行動
15  - **頻繁に購入される商品**: リピート率が高い商品を特定し、その商品に対するマーケティ...
16  - **セット販売の機会**: しばしば一緒に購入される商品のパターンを見つけ、バンドル販売...
17                          

In [6]:
# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

In [7]:
# ワークフロー化
print("処理を開始します。")

# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')

# 2. データをLLM用にテキスト形式に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"

# 3. OpenAI APIの呼び出し
# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"
# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])

# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

print("Excelファイルに分析結果を保存しました。")

処理を開始します。
Excelファイルに分析結果を保存しました。
